In [107]:
#Import libraries
os.chdir("/home/robbe/ionbot/")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re
import numpy as np
import os
from pathlib import Path
from pyteomics import mass as pymass
import OxiAnalysis as OA
import importlib
importlib.reload(OA)
import warnings; warnings.simplefilter('ignore')
import uniprot
import pprint

In [48]:
master = pd.read_csv("/home/robbe/ionbot/mastersets/master_first.csv")
masterpr = pd.read_csv("/home/robbe/ionbot/mastersets/PXD012477_first_proteins.csv")
mastermod = pd.read_csv("/home/robbe/ionbot/mastersets/PXD012477_modifications.csv")

In [49]:
#PXD006657 was still included in masterset but won't be used
master = master[master["Accession"] != 6657]
masterpr = masterpr[masterpr["Accession"] != 6657]
mastermod = mastermod[mastermod["Accession"] != 6657]

In [56]:
g = mastermod.groupby(["protein", "uniprot_id", "unexpected_modification", "position"])["#PSMs"].sum().to_frame().reset_index()
g

,protein,uniprot_id,unexpected_modification,position,#PSMs
0,1433B_HUMAN,P31946,[34]Methyl[N-TERM],63,1
1,1433B_HUMAN,P31946,[35]Oxidation[S],47,2
2,1433B_HUMAN,P31946,[35]Oxidation[Y],50,25
3,1433B_HUMAN,P31946,[35]oxidation[M],220,120
4,1433B_HUMAN,P31946,[4]Carbamidomethyl[Y],50,1
...,...,...,...,...,...
97997,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[99998]ICPL_2H(4)[K],258,1
97998,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[999992]GIST-Quat_2H(3)[N-TERM],279,3
97999,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[999993]GIST-Quat_2H(3)[K],258,13
98000,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[999999]Galactosyl[K],258,2


In [67]:
g10 = g[g["#PSMs"] > 10]

In [68]:
g10[g10["unexpected_modification"].isin(OA.modslist)]

,protein,uniprot_id,unexpected_modification,position,#PSMs
2,1433B_HUMAN,P31946,[35]Oxidation[Y],50,25
3,1433B_HUMAN,P31946,[35]oxidation[M],220,120
14,1433E_HUMAN,P62258,[35]Oxidation[Y],49,25
16,1433E_HUMAN,P62258,[35]oxidation[M],221,120
32,1433F_HUMAN,Q04917,[35]Oxidation[Y],49,25
...,...,...,...,...,...
97774,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[936]Chlorination[W],147,35
97775,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[936]Chlorination[W],283,52
97776,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[936]Chlorination[W],479,21
97777,sp|TRFE_HUMAN|,sp|TRFE_HUMAN|,[936]Chlorination[Y],155,14


Goal: per modification have a list of proteins, find out accessible surface of the modified residue, collect in a list

In [94]:
moddf = g10[g10["unexpected_modification"] == "[35]Oxidation[Y]"]
moddf[["uniprot_id", "position"]]

,uniprot_id,position
2,P31946,50
14,P62258,49
32,Q04917,49
40,P61981,49
54,P31947,48
...,...,...
97150,sp|TRFE_HUMAN|,536
97151,sp|TRFE_HUMAN|,578
97152,sp|TRFE_HUMAN|,593
97153,sp|TRFE_HUMAN|,669


In [102]:
moddict = dict()
for mod in OA.modslist:
    moddf = g10[g10["unexpected_modification"] == mod]
    #Remove contaminants
    moddf = moddf[~moddf["uniprot_id"].str.startswith("sp|")]
    protein_list = moddf["uniprot_id"].tolist()
    poslist = moddf["position"].tolist()
    pro_poslist = []
    for i in range(len(protein_list)):
        pro_pos = protein_list[i] + "|" + str(poslist[i])
        pro_poslist.append(pro_pos)
    moddict[mod] = pro_poslist

In [111]:
for key in moddict.keys():
    lijst = moddict[key]
    proteinlist = []
    for i in lijst:
        protein = i.split("|")[0]
        proteinlist.append(protein)
    uniprot_data = uniprot.batch_uniprot_metadata(proteinlist, 'cache')
    pprint.pprint(uniprot_data, indent=2)

Fetching metadata for 111 Uniprot IDs from http://uniprot.org ...
Error in fetching metadata
{}
Fetching metadata for 96 Uniprot IDs from http://uniprot.org ...
Error in fetching metadata
{}
Fetching metadata for 112 Uniprot IDs from http://uniprot.org ...
Error in fetching metadata
{}
Fetching metadata for 71 Uniprot IDs from http://uniprot.org ...
Error in fetching metadata
{}
Fetching metadata for 0 Uniprot IDs from http://uniprot.org ...
Error in fetching metadata
{}
Fetching metadata for 167 Uniprot IDs from http://uniprot.org ...
Error in fetching metadata
{}
Fetching metadata for 0 Uniprot IDs from http://uniprot.org ...
Error in fetching metadata
{}
Fetching metadata for 109 Uniprot IDs from http://uniprot.org ...
Error in fetching metadata
{}
Fetching metadata for 126 Uniprot IDs from http://uniprot.org ...
Error in fetching metadata
{}
Fetching metadata for 17 Uniprot IDs from http://uniprot.org ...
Error in fetching metadata
{}
Fetching metadata for 34 Uniprot IDs from http:

KeyboardInterrupt: 